<div style="background-color:#000;"><img src="pqn.png"></img></div>

In [ ]:
import os
from llama_index.llms.openai import OpenAI
from llama_index.core.workflow import Context
from llama_index.core.agent.workflow import (
    FunctionAgent,
    ReActAgent,
    AgentWorkflow,
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)
from tavily import AsyncTavilyClient
from dotenv import load_dotenv
load_dotenv()

llm = OpenAI(model="gpt-4o")

## Create a function to perform web searches asynchronously

We set up an asynchronous function to perform web searches using the Tavily API. This function will take a query string as input and return search results. It uses an API key from environment variables for authentication. The results are converted to a string format before being returned.

In [ ]:
async def search_web(query: str) -> str:
    client = AsyncTavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    return str(await client.search(query))

The function is designed to be asynchronous, allowing other tasks to run while waiting for the search results. It initializes an asynchronous client by using the API key stored in the environment variables, ensuring secure access. The search results are awaited, meaning the function will pause until the results are ready. The results are then converted to a string to ensure compatibility with other parts of the program.

## Implement functions to record, write, and review notes and reports

We define several asynchronous functions to record notes, write reports, and review them. Each function interacts with a context object to store and retrieve the current state. These functions update the state with relevant data, such as notes or report content, and return confirmation messages.

In [ ]:
async def record_notes(ctx: Context, notes: str, notes_title: str) -> str:
    current_state = await ctx.get("state")
    if "research_notes" not in current_state:
        current_state["research_notes"] = {}
    current_state["research_notes"][notes_title] = notes
    await ctx.set("state", current_state)
    return "Notes recorded."

In [ ]:
async def write_report(ctx: Context, report_content: str) -> str:
    current_state = await ctx.get("state")
    current_state["report_content"] = report_content
    await ctx.set("state", current_state)
    return "Report written."

In [ ]:
async def review_report(ctx: Context, review: str) -> str:
    current_state = await ctx.get("state")
    current_state["review"] = review
    await ctx.set("state", current_state)
    return "Report reviewed."

These functions perform different tasks related to managing information. The `record_notes` function checks if research notes exist in the current state and adds them if not. It then updates the notes with the provided title and content. The `write_report` function saves the report content to the state, while the `review_report` function saves feedback on the report. The context object ensures that the state is maintained consistently across different parts of the program.

## Define agents to handle specific tasks in the workflow

We create three different agents: ResearchAgent, WriteAgent, and ReviewAgent. Each agent has a specific purpose and is equipped with a set of functions it can use. The agents can hand off control to each other when needed to complete the workflow.

In [ ]:
research_agent = FunctionAgent(
    name="ResearchAgent",
    description="Useful for searching the web for information on a given topic and recording notes on the topic.",
    system_prompt=(
        "You are the ResearchAgent that can search the web for information on a given topic and record notes on the topic. "
        "Once notes are recorded and you are satisfied, you should hand off control to the WriteAgent to write a report on the topic. "
        "You should have at least some notes on a topic before handing off control to the WriteAgent."
    ),
    llm=llm,
    tools=[search_web, record_notes],
    can_handoff_to=["WriteAgent"],
)

In [ ]:
write_agent = FunctionAgent(
    name="WriteAgent",
    description="Useful for writing a report on a given topic.",
    system_prompt=(
        "You are the WriteAgent that can write a report on a given topic. "
        "Your report should be in a markdown format. The content should be grounded in the research notes. "
        "Once the report is written, you should get feedback at least once from the ReviewAgent."
    ),
    llm=llm,
    tools=[write_report],
    can_handoff_to=["ReviewAgent", "ResearchAgent"],
)

In [ ]:
review_agent = FunctionAgent(
    name="ReviewAgent",
    description="Useful for reviewing a report and providing feedback.",
    system_prompt=(
        "You are the ReviewAgent that can review the write report and provide feedback. "
        "Your review should either approve the current report or request changes for the WriteAgent to implement. "
        "If you have feedback that requires changes, you should hand off control to the WriteAgent to implement the changes after submitting the review."
    ),
    llm=llm,
    tools=[review_report],
    can_handoff_to=["WriteAgent"],
)

Each agent has a specific role to play in the workflow. The ResearchAgent is responsible for gathering information and recording notes. Once sufficient notes are gathered, it hands off control to the WriteAgent. The WriteAgent takes these notes and composes a report in markdown format. After writing, it seeks feedback from the ReviewAgent. The ReviewAgent evaluates the report, providing approval or suggesting changes. This structured approach ensures that tasks are completed efficiently and accurately.

## Set up and execute the workflow using the defined agents

We set up an AgentWorkflow to organize the sequence of actions performed by the agents. The workflow starts with the ResearchAgent and maintains a state to track progress. We then run the workflow with a user message, simulating a task request.

In [ ]:
agent_workflow = AgentWorkflow(
    agents=[research_agent, write_agent, review_agent],
    root_agent=research_agent.name,
    initial_state={
        "research_notes": {},
        "report_content": "Not written yet.",
        "review": "Review required.",
    },
)

In [ ]:
handler = agent_workflow.run(
    user_msg=(
        "Write me a report on why the announcement of the Deepseek R1 large language model rattled investors in Nvidia. "
        "Briefly describe what Deepseek R1 is, why it made news globally, and why Nvidia's stock price tanked. "
    )
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")

The workflow is initialized with a list of agents and their respective roles. The ResearchAgent is set as the starting point, guiding the initial data gathering. The initial state is defined to track the progress of notes, report content, and review status. The workflow is executed by running it with a user message, which simulates a request to perform a specific task. This setup ensures that the agents collaborate smoothly, following a structured sequence to achieve the desired outcome.

## Your next steps

Try modifying the code to explore different functionalities. You might change the user message to request a report on a different topic. Alternatively, consider integrating additional tools for the agents to use. This will help you understand the flexibility and potential of the agent-based workflow.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.